In [1]:
Output = c("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.3. Biomarker Distribution Analysis/Output")
cur_date = "061423"

library(readxl)
library(openxlsx)
library(tidyverse)
library(reshape2)
library(rlang)
library(PMCMRplus)

# reading in files
cytokine_df = data.frame(read_excel("Input/Cytokine_Data_050423.xlsx", sheet = 2))
proteomics_5_df = data.frame(read_excel("Input/Proteomics_Data_5ug_040423.xlsx", sheet = 2))
proteomics_25_df = data.frame(read_excel("Input/Proteomics_Data_25ug_040423.xlsx", sheet = 2))
mRNA_df = data.frame(read_excel("Input/Imputed_mRNA_Data_042623.xlsx"))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘rlang’


The following objects are masked from ‘package:purrr’:

    %@%, flatten, flatten_chr, flatten_dbl, flatten_int, flatten_lgl,
    flatten_raw, invoke, splice


Warning message:
“Expecting numeric in E1226 / R1226C5: got 'NA'”
Warning message:
“Expecting numeric in E1227 / R1227C5: got 'NA'”
Warning message:
“Expecting numeric in E1228 / R1228C5

In [2]:
head(cytokine_df)
head(proteomics_25_df)
head(mRNA_df)

,Subject_No,Subject_ID,Condensate,Burn_Condition,Condensate_Conc,Cytokine,Cytokine_Conc,Cytokine_Conc_pslog2
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
1,1,F_1,C,F,1,Eotaxin3,21.750958,4.507855
2,1,F_1,C,F,25,Eotaxin3,11.624224,3.658123
3,1,F_1,C,F,5,Eotaxin3,17.828714,4.234863
4,1,F_1,C,S,1,Eotaxin3,22.798534,4.572801
5,1,F_1,C,S,25,Eotaxin3,10.342567,3.503675
6,1,F_1,C,S,5,Eotaxin3,9.140652,3.342078


,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1488600000,30.47131
2,2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3806600000,31.82586
3,3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3900000000,31.86083
4,4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5373200000,32.32313
5,5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,2595200000,31.27320
6,6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5780700000,32.42860


,Subject_ID,Condensate,Burn_Condition,Concentration,Time_Point,mRNA,ddCT_pslog2
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,M_6,PBS,PBS,NA,24,HMOX1,4.871886
2,M_6,PBS,PBS,NA,24,ALDH3A1,4.837012
3,M_6,PBS,PBS,NA,24,CXCL1,4.822372
4,M_6,PBS,PBS,NA,24,CXCR1,3.940873
5,M_6,PBS,PBS,NA,24,GCLC,4.863186
6,M_6,PBS,PBS,NA,24,GCLM,4.834127


# Research Question: Are there statistically significant differences in each biomarker between burn conditions within the condensate samples?

Testing for statistical differences by comparing an individual biomarker's value (ie. concentration, intensity or ddCT) between burn conditions (ie. control, smoldering, and flaming) within each condensate (ie. plastic and cardboard) time point (ie. 4, 24, 72).

A Friedman test is a non-paramteric alternative to an ANOVA. A Nemenyi post-hoc will be subsequently run. https://www.rdocumentation.org/packages/PMCMR/versions/4.3/topics/posthoc.friedman.nemenyi.test
https://www.statology.org/friedman-test-r/


# Friedman's Test

In [3]:
friedmans_burn_condition = function(df, concentration, value, biomarker, biomarker_name){
    # """
    # Running Friedman's tests comparing burn conditions after filtering for biomarkers, condensate, concentration. 
    # Ultimately using this test to compare biomarker expression (control, smoldering, flaming).

    # :param: dataframe, concentration variable, value, biomarker name, biomarker variable name
    # :output: a dataframe containing the biomarker, variable name, condensate, concentration, time point, 
    # comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    unique_biomarker = unique(df[[biomarker_name]])
    unique_condensate = c("C", "P")
    unique_conc = na.omit(unique(df[[concentration]]))
    
    friedmans_df = data.frame()
    nemenyis_df = data.frame()
    # iterating through each concentration
    for(i in 1:length(unique_biomarker)){
        for(j in 1:length(unique_condensate)){
            for(k in 1:length(unique_conc)){
                  
                # filtering for each biomarker, conc
                filtered_df = df %>%
                    filter(eval(rlang::parse_expr(biomarker_name)) == unique_biomarker[i], Condensate == "PBS" |
                           Condensate == unique_condensate[j] &
                           eval(rlang::parse_expr(concentration)) == unique_conc[k])


                # friedman's
                friedmans_test = friedman.test(filtered_df[[value]], filtered_df$Burn_Condition, 
                                               filtered_df$Subject_ID)
                friedmans_stat_pvalue = c(friedmans_test$statistic, friedmans_test$p.value)


                # Nemenyi's post hoc
                nemenyis_test = frdAllPairsNemenyiTest(filtered_df[[value]], filtered_df$Burn_Condition, 
                                                       filtered_df$Subject_ID)
                nemenyis_stats = c(nemenyis_test$statistic[1], nemenyis_test$statistic[2], 
                                   nemenyis_test$statistic[4])
                nemenyis_pvalues = c(nemenyis_test$p.value[1], nemenyis_test$p.value[2], 
                                    nemenyis_test$p.value[4])
                nemenyis_comparison = c(paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                                rownames(nemenyis_test$statistic)[1]),
                                         paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                                rownames(nemenyis_test$statistic)[2]),
                                         paste0(colnames(nemenyis_test$statistic)[2], " vs. ", 
                                                rownames(nemenyis_test$statistic)[2]))


                # contains biomarker, biomarker name, condensate, conc, stat, and p value
                friedmans_vector = cbind(biomarker, unique_biomarker[i], unique_condensate[j], unique_conc[k], NA, 
                                         friedmans_test$statistic, friedmans_test$p.value)
                nemenyis_matrix = cbind(biomarker, unique_biomarker[i], unique_condensate[j], unique_conc[k], NA, 
                                         nemenyis_comparison, nemenyis_stats, nemenyis_pvalues)
                friedmans_df = rbind(friedmans_df, friedmans_vector)
                nemenyis_df = rbind(nemenyis_df, nemenyis_matrix)
                
            }
        }
    }
    
    # adding col names
    colnames(friedmans_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Statistic", "P Value")
    colnames(nemenyis_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Comparison", "Statistic", "P Value")
    
    # calculating padj values
    friedmans_df = friedmans_df %>%
        group_by(Condensate, Concentration) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))
    nemenyis_df = nemenyis_df %>%
        group_by(Condensate, Concentration) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))

    
    return(list(friedmans_df, nemenyis_df))
}

# calling fn
cytokine_friedmans = friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Cytokine_Conc_pslog2", "Cytokine",
                                             "Cytokine")
proteomics_5_friedmans = friedmans_burn_condition(proteomics_5_df, "Concentration","Intensity_pslog2", "Protein", 
                                                  "Protein_Accession")
proteomics_25_friedmans = friedmans_burn_condition(proteomics_25_df, "Concentration","Intensity_pslog2", 
                                                   "Protein", "Protein_Accession")

In [4]:
# the mRNA df doesn't consistently test all 3 concentrations (1,5, or 25 micrograms) or 
# time points(4,24,72) therefore we can't subset the df using a loop so we'll create separate dfs
# for each concentration and time point
split_mRNA_df = mRNA_df %>%
    group_by(Concentration, Time_Point) %>%
    group_split()

conc1_24_mRNA_df = split_mRNA_df[[1]]
conc1_72_mRNA_df = split_mRNA_df[[2]]
conc25_4_mRNA_df = split_mRNA_df[[3]]
conc25_24_mRNA_df = split_mRNA_df[[4]]
conc25_72_mRNA_df = split_mRNA_df[[5]]
conc5_24_mRNA_df = split_mRNA_df[[6]]
conc5_72_mRNA_df = split_mRNA_df[[7]]
concc_4_mRNA_df = split_mRNA_df[[8]]
concc_24_mRNA_df = split_mRNA_df[[9]]
concc_72_mRNA_df = split_mRNA_df[[10]]

# control samples have a concentration of NA, so they're all in a separate df
# adding them back into the other dataframes
conc1_24_mRNA_df = unique(rbind(conc1_24_mRNA_df, concc_24_mRNA_df))
conc1_72_mRNA_df = unique(rbind(conc1_72_mRNA_df, concc_72_mRNA_df))
conc25_4_mRNA_df = unique(rbind(conc25_4_mRNA_df, concc_4_mRNA_df))
conc25_24_mRNA_df = unique(rbind(conc25_24_mRNA_df, concc_24_mRNA_df))
conc25_72_mRNA_df = unique(rbind(conc25_72_mRNA_df, concc_72_mRNA_df))
conc5_24_mRNA_df = unique(rbind(conc5_24_mRNA_df, concc_24_mRNA_df))
conc5_72_mRNA_df = unique(rbind(conc5_72_mRNA_df, concc_72_mRNA_df))

head(conc1_24_mRNA_df)

Subject_ID,Condensate,Burn_Condition,Concentration,Time_Point,mRNA,ddCT_pslog2
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
M_6,P,F,1,24,HMOX1,4.774776
M_6,P,F,1,24,ALDH3A1,4.762372
M_6,P,F,1,24,CXCL1,4.768515
M_6,P,F,1,24,CXCR1,3.814665
M_6,P,F,1,24,GCLC,4.792038
M_6,P,F,1,24,GCLM,4.814864


In [5]:
friedmans_burn_condition_v2 = function(df, value, biomarker){
    # """
    # Running Friedman's tests comparing burn conditions after filtering for biomarkers and condensate. 
    # Ultimately using this test to compare biomarker expression (control, smoldering, flaming).

    # :param: dataframe, value, biomarker variable name
    # :output: a dataframe containing the biomarker, variable name, condensate, concentration, time point, 
    # comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    unique_biomarker = unique(df[[biomarker]])
    unique_condensate = c("C", "P")
    
    friedmans_df = data.frame()
    nemenyis_df = data.frame()
    # iterating through each concentration
    for(i in 1:length(unique_biomarker)){
        for(j in 1:length(unique_condensate)){
                  
            # filtering for each biomarker, conc
            filtered_df = df %>%
                filter(eval(rlang::parse_expr(biomarker)) == unique_biomarker[i], 
                       Condensate != unique_condensate[j])


            # friedman's
            friedmans_test = friedman.test(filtered_df[[value]], filtered_df$Burn_Condition, 
                                           filtered_df$Subject_ID)
            friedmans_stat_pvalue = c(friedmans_test$statistic, friedmans_test$p.value)


            # Nemenyi's post hoc
            nemenyis_test = frdAllPairsNemenyiTest(filtered_df[[value]], filtered_df$Burn_Condition, 
                                                   filtered_df$Subject_ID)
            nemenyis_stats = c(nemenyis_test$statistic[1], nemenyis_test$statistic[2], 
                               nemenyis_test$statistic[4])
            nemenyis_pvalues = c(nemenyis_test$p.value[1], nemenyis_test$p.value[2], 
                                nemenyis_test$p.value[4])
            nemenyis_comparison = c(paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                            rownames(nemenyis_test$statistic)[1]),
                                     paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                            rownames(nemenyis_test$statistic)[2]),
                                     paste0(colnames(nemenyis_test$statistic)[2], " vs. ", 
                                            rownames(nemenyis_test$statistic)[2]))


            # contains biomarker, biomarker name, conc, time point, condensate, stat, and p value
            friedmans_vector = cbind(biomarker, unique_biomarker[i], unique_condensate[j], 
                                     unique(filtered_df$Concentration)[1], unique(filtered_df$Time_Point),
                                     friedmans_test$statistic, friedmans_test$p.value)
            nemenyis_matrix = cbind(biomarker, unique_biomarker[i], unique_condensate[j],
                                    unique(filtered_df$Concentration)[1], unique(filtered_df$Time_Point),
                                     nemenyis_comparison, nemenyis_stats, nemenyis_pvalues)
            friedmans_df = rbind(friedmans_df, friedmans_vector)
            nemenyis_df = rbind(nemenyis_df, nemenyis_matrix)

        }
    }
    
    # adding col names
    colnames(friedmans_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Statistic", "P Value")
    colnames(nemenyis_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Comparison", "Statistic", "P Value")
    
    # calculating padj values
    friedmans_df = friedmans_df %>%
        group_by(Condensate) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))
    nemenyis_df = nemenyis_df %>%
        group_by(Condensate) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))

    
    return(list(friedmans_df, nemenyis_df))
}

# calling fn

conc1_24_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc1_24_mRNA_df, "ddCT_pslog2", "mRNA")
conc1_72_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc1_72_mRNA_df, "ddCT_pslog2", "mRNA")
conc25_4_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc25_4_mRNA_df, "ddCT_pslog2", "mRNA")
conc25_24_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc25_24_mRNA_df, "ddCT_pslog2", "mRNA")
conc25_72_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc25_72_mRNA_df, "ddCT_pslog2", "mRNA")
conc5_24_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc5_24_mRNA_df, "ddCT_pslog2", "mRNA")
conc5_72_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc5_72_mRNA_df, "ddCT_pslog2", "mRNA")

In [6]:
# creating 1 df
friedmans_df = rbind(cytokine_friedmans[[1]], proteomics_25_friedmans[[1]], proteomics_5_friedmans[[1]],
                              conc1_24_friedmans_mRNA_v2[[1]], conc1_72_friedmans_mRNA_v2[[1]], conc25_4_friedmans_mRNA_v2[[1]], 
                              conc25_24_friedmans_mRNA_v2[[1]], conc25_72_friedmans_mRNA_v2[[1]], conc5_24_friedmans_mRNA_v2[[1]],
                              conc5_72_friedmans_mRNA_v2[[1]])
wilcoxon_df = rbind(cytokine_friedmans[[2]], proteomics_25_friedmans[[2]], proteomics_5_friedmans[[2]],
                              conc1_24_friedmans_mRNA_v2[[2]], conc1_72_friedmans_mRNA_v2[[2]], conc25_4_friedmans_mRNA_v2[[2]], 
                              conc25_24_friedmans_mRNA_v2[[2]], conc25_72_friedmans_mRNA_v2[[2]], conc5_24_friedmans_mRNA_v2[[2]],
                              conc5_72_friedmans_mRNA_v2[[2]])

head(friedmans_df)
head(wilcoxon_df)

Biomarker,Variable Name,Condensate,Concentration,Time Point,Statistic,P Value,P Adj
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Cytokine,Eotaxin3,C,1,NA,1.33333333333333,0.513417119032592,0.6713916
Cytokine,Eotaxin3,C,25,NA,0.333333333333333,0.846481724890614,1.0000000
Cytokine,Eotaxin3,C,5,NA,0.333333333333333,0.846481724890614,0.9593460
Cytokine,Eotaxin3,P,1,NA,2.33333333333333,0.311403223914598,0.4812595
Cytokine,Eotaxin3,P,25,NA,1,0.606530659712633,0.8464817
Cytokine,Eotaxin3,P,5,NA,2.33333333333333,0.311403223914598,0.4072196


Biomarker,Variable Name,Condensate,Concentration,Time Point,Comparison,Statistic,P Value,P Adj
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Cytokine,Eotaxin3,C,1,NA,F vs. PBS,0.816496580927726,0.832229218116194,0.9940869
Cytokine,Eotaxin3,C,1,NA,F vs. S,0.816496580927726,0.832229218116194,0.9940869
Cytokine,Eotaxin3,C,1,NA,PBS vs. S,1.63299316185545,0.480432575468159,0.9940869
Cytokine,Eotaxin3,C,25,NA,F vs. PBS,0.408248290463863,0.955103094750503,1.0000000
Cytokine,Eotaxin3,C,25,NA,F vs. S,0.408248290463863,0.955103094750503,1.0000000
Cytokine,Eotaxin3,C,25,NA,PBS vs. S,0.816496580927726,0.832229218116194,1.0000000


In [7]:
# exporting
write.xlsx(friedmans_df, paste0(Output,"/", "Friedmans_Biomarker_Results_", cur_date, ".xlsx"), 
           rowNames = FALSE)
write.xlsx(wilcoxon_df, paste0(Output,"/", "Wilcoxon_PostHoc_Biomarker_Results", cur_date, ".xlsx"), 
           rowNames = FALSE)